# Automated People Counting at Szabist University's 100 Campus 

#### This project revolves around the automation of people counting at the entrance of Szabist University's 100 Campus Building. Utilizing YOLOv8, a state-of-the-art object detection model, and CV Zone, a computer vision library, the system tracks individuals entering and exiting the campus gate.

In [3]:
!pip install cvzone

You should consider upgrading via the 'c:\users\d e l l\myenv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
#installing ultralytics to import YOLO
!pip install ultralytics

You should consider upgrading via the 'c:\users\d e l l\myenv\scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install filterpy

You should consider upgrading via the 'c:\users\d e l l\myenv\scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install scikit-image

You should consider upgrading via the 'c:\users\d e l l\myenv\scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
!pip install lap

You should consider upgrading via the 'c:\users\d e l l\myenv\scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip list

Package                   Version
------------------------- --------------------
anyio                     4.0.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.1
backcall                  0.2.0
beautifulsoup4            4.12.2
bleach                    6.1.0
certifi                   2023.7.22
cffi                      1.16.0
charset-normalizer        3.3.2
colorama                  0.4.6
comm                      0.2.0
contourpy                 1.1.1
cvzone                    1.6.1
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
exceptiongroup            1.1.3
executing                 2.0.1
fastjsonschema            2.18.1
filelock                  3.13.1
filterpy                  1.4.5
fonttools                 4.44.0
fsspec                    2023.10.0
idna 

You should consider upgrading via the 'c:\users\d e l l\myenv\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
#Libraries
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import cv2
import cvzone
import math
import lap
from filterpy.kalman import KalmanFilter
from skimage import io
from scipy.optimize import linear_sum_assignment
import sort
from sort import *
#from sort import *

#For Video
cap = cv2.VideoCapture("ProjectVID-20231111-WA0004.mp4")

#Model Path
model = YOLO("yolov8n.pt")

#Model names
#Only Detecting People
classNames = ['Person']

mask = cv2.imread("mask.png")

#Tracking
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limitIn = [300, 350,600, 350]
limitOut = [00, 350,200, 350]

totalCountIn = []
totalCountOut = []


while True:
    success, img = cap.read()
    #Overlay mask
    imgRegion = cv2.bitwise_and(img,mask)

    imgGraphics = cv2.imread("graphics.png",cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (0,0))
    
    results = model(imgRegion, stream=True) 

    detections = np.empty((0,5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img,  (x1, y1, w, h))
            #Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100     #We can use ceiling or floor
            #class name
            cls = int(box.cls[0])
            #currentClass = classNames[cls]

            #if currentClass == "person" and conf > 0.3:
            currentArray = np.array([x1,y1,x2,y2,conf])
            detections = np.vstack((detections,currentArray))
            
            #cvzone.putTextRect(img, f'{classNames[cls]} {conf}',(max(0, x1), max(35, y1)),scale=1, thickness=1, offset=5)
            
            resultsTracker = tracker.update(detections)

            cv2.line(img, (limitIn[0],limitIn[1]), (limitIn[2],limitIn[3]), (0,0,255), 3)
            cv2.line(img, (limitOut[0],limitOut[1]), (limitOut[2],limitOut[3]), (0,0,255), 3)
            
            for results in resultsTracker:
                x1,y1,x2,y2,id = results
                x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)
                print(results)
                w, h = x2 - x1, y2 - y1
                cvzone.cornerRect(img,(x1, y1, w, h), l=4, rt=1, colorR=(255,0,0))
                cvzone.putTextRect(img, f'{int(id)}',(max(0, x1), max(35, y1)),scale=1, thickness=2, offset=3)

                cx, cy = x1 + w // 2, y1 + h // 2
                cv2.circle(img,(cx,cy),5,(255,0,0),cv2.FILLED)

                if limitIn[0] < cx < limitIn[2] and limitIn[1] - 25 < cy < limitIn[1] + 25:
                    if totalCountIn.count(id) == 0:
                        totalCountIn.append(id)
                        cv2.line(img, (limitIn[0],limitIn[1]), (limitIn[2],limitIn[3]), (0,255,0), 3)


                if limitOut[0] < cx < limitOut[2] and limitOut[1] - 25 < cy < limitOut[1] + 25:
                    if totalCountOut.count(id) == 0:
                        totalCountOut.append(id)
                        cv2.line(img, (limitOut[0],limitOut[1]), (limitOut[2],limitOut[3]), (0,255,0), 3)
                        

    #cvzone.putTextRect(img, f'Count: {len(totalCount)}', (50, 50))
    cv2.putText(img,str(len(totalCountIn)),(360,75),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),7)
    cv2.putText(img,str(len(totalCountOut)),(150,75),cv2.FONT_HERSHEY_PLAIN,3,(0,0,255),7)

    
    cv2.imshow("Image",img)     #This code will run our webcam
   # cv2.imshow("ImageRegion",imgRegion)
    cv2.waitKey(1)              #We wanted to give it 1ms delay

[        314         189         438         555           1]
[          0         239          69         483           2]
[        314         189         438         555           1]
[         51         214         133         431           3]
[          0         239          69         483           2]
[        314         189         438         555           1]
[        314         189         438         555           1]


0: 640x384 4 persons, 1636.7ms
Speed: 40.0ms preprocess, 1636.7ms inference, 16.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 persons, 1231.3ms
Speed: 39.0ms preprocess, 1231.3ms inference, 16.1ms postprocess per image at shape (1, 3, 640, 384)


[     313.71      189.32      436.73       553.9           1]
[     313.73      189.48      436.24      553.51           1]
[     313.81      189.54      436.03      553.38           1]
[     48.627      216.81      128.72      432.63           3]
[     313.89      189.54      435.93      553.36           1]



0: 640x384 4 persons, 1030.8ms
Speed: 47.0ms preprocess, 1030.8ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 384)


[     313.46      189.74       434.3      552.49           1]
[     313.49      189.88      433.55      552.11           1]
[     313.63      189.95      433.17      551.96           1]
[     42.859      218.04      125.33      436.46           3]
[     313.77      189.97      432.97      551.92           1]


...

0: 640x384 3 persons, 906.5ms
Speed: 14.0ms preprocess, 906.5ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 384)



[    -1.6145      233.12      65.281      458.61          85]
[     -1.452      234.03      64.352      458.74          85]
[    -1.1835      234.51      63.823      458.63          85]


0: 640x384 3 persons, 1 handbag, 860.5ms
Speed: 13.0ms preprocess, 860.5ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 384)


[    -1.3177      230.44      62.582      456.95          85]
[     -1.158      228.44       61.93      456.75          85]
[   -0.93936      227.34      61.554      457.01          85]
[     78.692      238.31       175.6      438.62          96]
[    -0.7377      226.68       61.32      457.37          85]


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'
